In [1]:
%matplotlib inline

In [3]:
import argparse, time

import numpy as np
import mxnet as mx

from mxnet import gluon, nd
from mxnet import autograd as ag
from mxnet.gluon import nn

from gluoncv.model_zoo import get_model
from gluoncv.utils import makedirs, TrainingHistory

In [ ]:
# number of GPUs to use
num_gpus = 1
ctx = [mx.gpu(i) for i in range(num_gpus)]
b_size = 32
# Get the model ResNet50_v2, with 10 output classes
# net = get_model('ResNet50_v2', classes=1000)
# net.initialize(mx.init.MSRAPrelu(), ctx = ctx)

In [ ]:
jitter_param = 0.4
lighting_param = 0.1
mean_rgb = [123.68, 116.779, 103.939]
std_rgb = [58.393, 57.12, 57.375]

train_data = mx.io.ImageRecordIter(
    path_imgrec         = './data/rec/train.rec',
    path_imgidx         = './data/rec/train.idx',
    preprocess_threads  = 32,
    shuffle             = True,
    batch_size          = b_size,

    data_shape          = (3, 224, 224),
    mean_r              = mean_rgb[0],
    mean_g              = mean_rgb[1],
    mean_b              = mean_rgb[2],
    std_r               = std_rgb[0],
    std_g               = std_rgb[1],
    std_b               = std_rgb[2],
    rand_mirror         = True,
    random_resized_crop = True,
    max_aspect_ratio    = 4. / 3.,
    min_aspect_ratio    = 3. / 4.,
    max_random_area     = 1,
    min_random_area     = 0.08,
    brightness          = jitter_param,
    saturation          = jitter_param,
    contrast            = jitter_param,
    pca_noise           = lighting_param,
)

In [ ]:
val_data = mx.io.ImageRecordIter(
    path_imgrec         = './data/rec/val.rec',
    path_imgidx         = './data/rec/val.idx',
    preprocess_threads  = 32,
    shuffle             = False,
    batch_size          = b_size,

    resize              = 256,
    data_shape          = (3, 224, 224),
    mean_r              = mean_rgb[0],
    mean_g              = mean_rgb[1],
    mean_b              = mean_rgb[2],
    std_r               = std_rgb[0],
    std_g               = std_rgb[1],
    std_b               = std_rgb[2],
)

In [ ]:
# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements.  See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership.  The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License.  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied.  See the License for the
# specific language governing permissions and limitations
# under the License.


__all__ = ['EdgeLiteNet', 'edgeLiteNet']

from mxnet.context import cpu
from mxnet.gluon.block import HybridBlock
from mxnet.gluon import nn
from mxnet.gluon.nn import BatchNorm
from mxnet.gluon.contrib.nn import HybridConcurrent

def _make_basic_conv(in_channels, channels, norm_layer=BatchNorm, norm_kwargs=None, **kwargs):
    out = nn.HybridSequential(prefix='')
    out.add(nn.Conv2D(in_channels=in_channels, channels=channels, use_bias=False, **kwargs))
    out.add(norm_layer(in_channels=channels, epsilon=0.001,
                       **({} if norm_kwargs is None else norm_kwargs)))
    out.add(nn.Activation('relu'))
    return out

def _make_branch(use_pool, norm_layer, norm_kwargs, *conv_settings):
    out = nn.HybridSequential(prefix='')
    if use_pool == 'avg':
        out.add(nn.AvgPool2D(pool_size=3, strides=1, padding=1))
    elif use_pool == 'max':
        out.add(nn.MaxPool2D(pool_size=3, strides=1, padding=1))
    setting_names = ['in_channels', 'channels', 'kernel_size', 'strides', 'padding']
    for setting in conv_settings:
        kwargs = {}
        for i, value in enumerate(setting):
            if value is not None:
                if setting_names[i] == 'in_channels':
                    in_channels = value
                elif setting_names[i] == 'channels':
                    channels = value
                else:
                    kwargs[setting_names[i]] = value
        out.add(_make_basic_conv(in_channels, channels, norm_layer, norm_kwargs, **kwargs))
    return out

def _make_Mixed_4a(in_channels, pool_features, prefix, norm_layer, norm_kwargs):
    out = HybridConcurrent(axis=1, prefix=prefix)
    with out.name_scope():
        out.add(_make_branch(None, norm_layer, norm_kwargs,
                             (in_channels, 192, 1, None, None)))
        out.add(_make_branch(None, norm_layer, norm_kwargs,
                             (in_channels, 96, 1, None, None),
                             (96, 208, 3, None, 1)))
        out.add(_make_branch(None, norm_layer, norm_kwargs,
                             (in_channels, 16, 1, None, None),
                             (16, 48, 3, None, 1)))
        out.add(_make_branch('max', norm_layer, norm_kwargs,
                             (in_channels, pool_features, 1, None, None)))
    return out

def _make_Mixed_4b(in_channels, pool_features, prefix, norm_layer, norm_kwargs):
    out = HybridConcurrent(axis=1, prefix=prefix)
    with out.name_scope():
        out.add(_make_branch(None, norm_layer, norm_kwargs,
                             (in_channels, 160, 1, None, None)))
        out.add(_make_branch(None, norm_layer, norm_kwargs,
                             (in_channels, 112, 1, None, None),
                             (112, 224, 3, None, 1)))
        out.add(_make_branch(None, norm_layer, norm_kwargs,
                             (in_channels, 24, 1, None, None),
                             (24, 64, 3, None, 1)))
        out.add(_make_branch('max', norm_layer, norm_kwargs,
                             (in_channels, pool_features, 1, None, None)))
    return out

def _make_Mixed_4c(in_channels, pool_features, prefix, norm_layer, norm_kwargs):
    out = HybridConcurrent(axis=1, prefix=prefix)
    with out.name_scope():
        out.add(_make_branch(None, norm_layer, norm_kwargs,
                             (in_channels, 128, 1, None, None)))
        out.add(_make_branch(None, norm_layer, norm_kwargs,
                             (in_channels, 128, 1, None, None),
                             (128, 256, 3, None, 1)))
        out.add(_make_branch(None, norm_layer, norm_kwargs,
                             (in_channels, 24, 1, None, None),
                             (24, 64, 3, None, 1)))
        out.add(_make_branch('max', norm_layer, norm_kwargs,
                             (in_channels, pool_features, 1, None, None)))
    return out

def _make_Mixed_4d(in_channels, pool_features, prefix, norm_layer, norm_kwargs):
    out = HybridConcurrent(axis=1, prefix=prefix)
    with out.name_scope():
        out.add(_make_branch(None, norm_layer, norm_kwargs,
                             (in_channels, 112, 1, None, None)))
        out.add(_make_branch(None, norm_layer, norm_kwargs,
                             (in_channels, 144, 1, None, None),
                             (144, 288, 3, None, 1)))
        out.add(_make_branch(None, norm_layer, norm_kwargs,
                             (in_channels, 32, 1, None, None),
                             (32, 64, 3, None, 1)))
        out.add(_make_branch('max', norm_layer, norm_kwargs,
                             (in_channels, pool_features, 1, None, None)))
    return out

def _make_Mixed_4e(in_channels, pool_features, prefix, norm_layer, norm_kwargs):
    out = HybridConcurrent(axis=1, prefix=prefix)
    with out.name_scope():
        out.add(_make_branch(None, norm_layer, norm_kwargs,
                             (in_channels, 256, 1, None, None)))
        out.add(_make_branch(None, norm_layer, norm_kwargs,
                             (in_channels, 160, 1, None, None),
                             (160, 320, 3, None, 1)))
        out.add(_make_branch(None, norm_layer, norm_kwargs,
                             (in_channels, 32, 1, None, None),
                             (32, 128, 3, None, 1)))
        out.add(_make_branch('max', norm_layer, norm_kwargs,
                             (in_channels, pool_features, 1, None, None)))
    return out

def _make_Mixed_5a(in_channels, pool_features, prefix, norm_layer, norm_kwargs):
    out = HybridConcurrent(axis=1, prefix=prefix)
    with out.name_scope():
        out.add(_make_branch(None, norm_layer, norm_kwargs,
                             (in_channels, 256, 1, None, None)))
        out.add(_make_branch(None, norm_layer, norm_kwargs,
                             (in_channels, 160, 1, None, None),
                             (160, 320, 3, None, 1)))
        out.add(_make_branch(None, norm_layer, norm_kwargs,
                             (in_channels, 32, 1, None, None),
                             (32, 128, 3, None, 1)))
        out.add(_make_branch('max', norm_layer, norm_kwargs,
                             (in_channels, pool_features, 1, None, None)))
    return out

def _make_Mixed_5b(in_channels, pool_features, prefix, norm_layer, norm_kwargs):
    out = HybridConcurrent(axis=1, prefix=prefix)
    with out.name_scope():
        out.add(_make_branch(None, norm_layer, norm_kwargs,
                             (in_channels, 384, 1, None, None)))
        out.add(_make_branch(None, norm_layer, norm_kwargs,
                             (in_channels, 192, 1, None, None),
                             (192, 384, 3, None, 1)))
        out.add(_make_branch(None, norm_layer, norm_kwargs,
                             (in_channels, 48, 1, None, None),
                             (48, 128, 3, None, 1)))
        out.add(_make_branch('max', norm_layer, norm_kwargs,
                             (in_channels, pool_features, 1, None, None)))
    return out

def _make_aux(in_channels, classes, norm_layer, norm_kwargs):
    out = nn.HybridSequential(prefix='')
    out.add(nn.AvgPool2D(pool_size=5, strides=3))
    out.add(_make_basic_conv(in_channels=in_channels, channels=128, kernel_size=1,
                             norm_layer=norm_layer, norm_kwargs=norm_kwargs))

    out.add(nn.Flatten())
    out.add(nn.Dense(units=1024, in_units=2048))
    out.add(nn.Activation('relu'))
    out.add(nn.Dropout(0.7))
    out.add(nn.Dense(units=classes, in_units=1024))
    return out

class EdgeLiteNet(HybridBlock):

    def __init__(self, classes=1000, norm_layer=BatchNorm, dropout_ratio=0.4, aux_logits=False,
                 norm_kwargs=None, partial_bn=False, pretrained_base=True, ctx=None, **kwargs):
        super(EdgeLiteNet, self).__init__(**kwargs)
        self.dropout_ratio = dropout_ratio
        self.aux_logits = aux_logits

        with self.name_scope():
            self.conv1 = _make_basic_conv(in_channels=3, channels=64, kernel_size=7,
                                          strides=2, padding=3,
                                          norm_layer=norm_layer, norm_kwargs=norm_kwargs)
            self.maxpool1 = nn.MaxPool2D(pool_size=3, strides=2, ceil_mode=True)

            if partial_bn:
                if norm_kwargs is not None:
                    norm_kwargs['use_global_stats'] = True
                else:
                    norm_kwargs = {}
                    norm_kwargs['use_global_stats'] = True

            self.conv2 = _make_basic_conv(in_channels=64, channels=64, kernel_size=1,
                                          norm_layer=norm_layer, norm_kwargs=norm_kwargs)
            self.conv3 = _make_basic_conv(in_channels=64, channels=192,
                                          kernel_size=3, padding=1,
                                          norm_layer=norm_layer, norm_kwargs=norm_kwargs)
            self.maxpool2 = nn.MaxPool2D(pool_size=3, strides=2, ceil_mode=True)

            self.edgelite4a = _make_Mixed_4a(192, 64, 'Mixed_4a_', norm_layer, norm_kwargs)
            self.edgelite4b = _make_Mixed_4b(512, 64, 'Mixed_4b_', norm_layer, norm_kwargs)
            self.edgelite4c = _make_Mixed_4c(512, 64, 'Mixed_4c_', norm_layer, norm_kwargs)
            self.edgelite4d = _make_Mixed_4d(512, 64, 'Mixed_4d_', norm_layer, norm_kwargs)
            self.edgelite4e = _make_Mixed_4e(528, 128, 'Mixed_4e_', norm_layer, norm_kwargs)
            self.maxpool4 = nn.MaxPool2D(pool_size=2, strides=2)

            self.edgelite5a = _make_Mixed_5a(832, 128, 'Mixed_5a_', norm_layer, norm_kwargs)
            self.edgelite5b = _make_Mixed_5b(832, 128, 'Mixed_5b_', norm_layer, norm_kwargs)

            if self.aux_logits:
                self.aux1 = _make_aux(512, classes, norm_layer, norm_kwargs)
                self.aux2 = _make_aux(528, classes, norm_layer, norm_kwargs)

            self.head = nn.HybridSequential(prefix='')
            self.avgpool = nn.AvgPool2D(pool_size=7)
            self.dropout = nn.Dropout(self.dropout_ratio)
            #self.dropout = nn.Dropout(self.dropout_ratio)
            self.output = nn.Dense(units=classes, in_units=4096)
            #self.output = nn.Dense(units=classes, in_units=1024)
            self.head.add(self.avgpool)
            self.head.add(self.dropout)
            self.head.add(self.output)
    def hybrid_forward(self, F, x):
        x = self.conv1(x)
        x = self.maxpool1(x)

        x = self.conv2(x)
        x = self.conv3(x)
        x = self.maxpool2(x)

        x = self.edgelite4a(x)
        if self.aux_logits:
            aux1 = self.aux1(x)
        x = self.edgelite4b(x)
        x = self.edgelite4c(x)
        x = self.edgelite4d(x)
        if self.aux_logits:
            aux2 = self.aux2(x)
        x = self.edgelite4e(x)
        x = self.maxpool4(x)

        x = self.edgelite5a(x)
        x = self.edgelite5b(x)
        x = self.head(x)

        if self.aux_logits:
            return (x, aux2, aux1)
        return x


def edgeLiteNet(classes=1000, pretrained=False, pretrained_base=True, ctx=cpu(),
              dropout_ratio=0.4, aux_logits=False,
              root='~/.mxnet/models', partial_bn=False, **kwargs):
    net = EdgeLiteNet(classes=classes, partial_bn=partial_bn, pretrained_base=pretrained_base,
                    dropout_ratio=dropout_ratio, aux_logits=aux_logits, ctx=ctx, **kwargs)
    if pretrained:
        from .model_store import get_model_file
        net.load_parameters(get_model_file('edgeLiteNet',
                                           tag=pretrained, root=root), ctx=ctx, cast_dtype=True)
        from ..data import ImageNet1kAttr
        attrib = ImageNet1kAttr()
        net.synset = attrib.synset
        net.classes = attrib.classes
        net.classes_long = attrib.classes_long
    return net

In [ ]:
net = edgeLiteNet(classes=1000)
# Initialize the parameters with Xavier initializer
net.collect_params().initialize(mx.init.Xavier(), ctx=ctx)
print(net)

In [ ]:
# Learning rate decay factor
lr_decay = 0.1
# Epochs where learning rate decays
lr_decay_epoch = [30, 60, 90, np.inf]

# Nesterov accelerated gradient descent
optimizer = 'nag'
# Set parameters
optimizer_params = {'learning_rate': 0.1, 'wd': 0.0001, 'momentum': 0.9}

# Define our trainer for net
trainer = gluon.Trainer(net.collect_params(), optimizer, optimizer_params)

In [ ]:
L = gluon.loss.SoftmaxCrossEntropyLoss()

In [ ]:
acc_top1 = mx.metric.Accuracy()
acc_top5 = mx.metric.TopKAccuracy(5)
train_history = TrainingHistory(['training-top1-err', 'training-top5-err',
                                 'validation-top1-err', 'validation-top5-err'])

In [ ]:
def test(ctx, val_data):
    acc_top1_val = mx.metric.Accuracy()
    acc_top5_val = mx.metric.TopKAccuracy(5)
    for i, batch in enumerate(val_data):
        data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
        label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)
        outputs = [net(X) for X in data]
        acc_top1_val.update(label, outputs)
        acc_top5_val.update(label, outputs)

    _, top1 = acc_top1_val.get()
    _, top5 = acc_top5_val.get()
    return (1 - top1, 1 - top5)


In [ ]:
epochs = 1
lr_decay_count = 0
log_interval = 50
lr_decay_period = 0
batch_size = b_size

net.hybridize()

for epoch in range(epochs):
    tic = time.time()
    btic = time.time()
    acc_top1.reset()
    acc_top5.reset()

    if lr_decay_period == 0 and epoch == lr_decay_epoch[lr_decay_count]:
        trainer.set_learning_rate(trainer.learning_rate*lr_decay)
        lr_decay_count += 1

    for i, batch in enumerate(train_data):
        data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
        label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)
        with ag.record():
#             for X in data:
#                 print (X)
            outputs = [net(X) for X in data]
            loss = [L(yhat, y) for yhat, y in zip(outputs, label)]
        ag.backward(loss)
        trainer.step(batch_size, ignore_stale_grad=True)
        acc_top1.update(label, outputs)
        acc_top5.update(label, outputs)
        if log_interval and not (i + 1) % log_interval:
            _, top1 = acc_top1.get()
            _, top5 = acc_top5.get()
            err_top1, err_top5 = (1-top1, 1-top5)
            print('Epoch[%d] Batch [%d]     Speed: %f samples/sec   top1-err=%f     top5-err=%f'%(
                      epoch, i, batch_size/(time.time()-btic), err_top1, err_top5))
            btic = time.time()

    _, top1 = acc_top1.get()
    _, top5 = acc_top5.get()
    err_top1, err_top5 = (1-top1, 1-top5)

    err_top1_val, err_top5_val = test(ctx, val_data)
    train_history.update([err_top1, err_top5, err_top1_val, err_top5_val])

    print('[Epoch %d] training: err-top1=%f err-top5=%f'%(epoch, err_top1, err_top5))
    print('[Epoch %d] time cost: %f'%(epoch, time.time()-tic))
    print('[Epoch %d] validation: err-top1=%f err-top5=%f'%(epoch, err_top1_val, err_top5_val))

In [ ]:
#Save net parameters
current_model = 'edgeLite'
# net1 = gluon.nn.HybridSequential()
# net1.hybridize()
# with net1.name_scope():
#     net1.add(net)

net.export(current_model, 10)
saved_model = current_model + '.params'
print (saved_model)
net.save_parameters(saved_model)

In [ ]:
# Downloaded input symbol and params files
sym = './edgeLite-symbol.json'
params = './edgeLite-0010.params'

# Standard Imagenet input - 3 channels, 224*224
input_shape = (1,3,224,224)

# Path of the output file
onnx_file = './edgeLite.onnx'

In [ ]:
import mxnet.contrib.onnx as onnx_mxnet
converted_model_path = onnx_mxnet.export_model(sym, params, [input_shape], np.float32, onnx_file)

In [ ]:
def transform(image, label):
    resized = mx.image.resize_short(image, EDGE)
    cropped, crop_info = mx.image.center_crop(resized, SIZE)
    transposed = nd.transpose(cropped, (2,0,1))
    return transposed, label

In [ ]:
#training data path of hazard dataset
training_path = "./data/101_ObjectCategories"
print(training_path)
#testing data path of hazard dataset
testing_path = "./data/101_ObjectCategories_test"
print(testing_path)

In [ ]:
from mxnet.gluon.data.vision.datasets import ImageFolderDataset
dataset_train = ImageFolderDataset(root=training_path)
dataset_test = ImageFolderDataset(root=testing_path)

In [ ]:
EDGE = 224
SIZE = (EDGE, EDGE)
BATCH_SIZE = b_size
NUM_WORKERS = 6

In [ ]:
from mxnet.gluon.data import DataLoader

dataloader_train_ft = DataLoader(dataset_train.transform(transform, lazy=False), batch_size=BATCH_SIZE, last_batch='rollover',
                              shuffle=True, num_workers=NUM_WORKERS)
dataloader_test_ft = DataLoader(dataset_test.transform(transform, lazy=False), batch_size=BATCH_SIZE, last_batch='rollover',
                             shuffle=False, num_workers=NUM_WORKERS)
print("Train dataset: {} images, Test dataset: {} images".format(len(dataset_train), len(dataset_test)))

In [ ]:
categories = dataset_train.synsets
NUM_CLASSES = len(categories)
BATCH_SIZE = b_size
#print(NUM_CLASSES)

In [ ]:
import matplotlib.pyplot as plt

N = 100
plt.imshow((transform(dataset_train[N][0], 0)[0].asnumpy().transpose((1,2,0))))
plt.axis('off')
print(categories[dataset_train[N][1]])

In [ ]:
sym, arg_params, aux_params = onnx_mxnet.import_model('edgeLite.onnx')
def get_layer_output(symbol, arg_params, aux_params, layer_name):
    all_layers = symbol.get_internals()
    net = all_layers[layer_name+'_output']
    net = mx.symbol.Flatten(data=net)
    new_args = dict({k:arg_params[k] for k in arg_params if k in net.list_arguments()})
    new_aux = dict({k:aux_params[k] for k in aux_params if k in net.list_arguments()})
    return (net, new_args, new_aux)

sym.get_internals()

In [ ]:
new_sym, new_arg_params, new_aux_params = get_layer_output(sym, arg_params, aux_params, 'flatten0')

In [ ]:
ctx = mx.gpu() if mx.context.num_gpus() > 0 else mx.cpu()

In [ ]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    pre_trained = gluon.nn.SymbolBlock(outputs=new_sym, inputs=mx.sym.var('data'))

net_params = pre_trained.collect_params()

for param in new_arg_params:
    if param in net_params:
        net_params[param]._load_init(new_arg_params[param], ctx=ctx)
for param in new_aux_params:
    if param in net_params:
        net_params[param]._load_init(new_aux_params[param], ctx=ctx)

In [ ]:
#add new layer for fine-tuning
NUM_CLASSES_NEW = 2
dense_layer = gluon.nn.Dense(NUM_CLASSES_NEW)
dense_layer.initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)

In [ ]:
net = gluon.nn.HybridSequential()
net.hybridize()
with net.name_scope():
    net.add(pre_trained)
    net.add(dense_layer)

In [ ]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

In [ ]:
LEARNING_RATE = 0.001
WDECAY = 0.0005
MOMENTUM = 0.9

In [ ]:
ctx = mx.gpu() 
net.collect_params().initialize(ctx=ctx)
trainer = gluon.Trainer(net.collect_params(), 'sgd', 
                        {'learning_rate': LEARNING_RATE,
                         'wd':WDECAY,
                         'momentum':MOMENTUM})

In [ ]:
def evaluate_accuracy_gluon(data_iterator, net):
    num_instance = 0
    sum_metric = nd.zeros(1,ctx=ctx, dtype=np.int32)
    for i, (data, label) in enumerate(data_iterator):
        data = data.astype(np.float32).as_in_context(ctx)
        label = label.astype(np.int32).as_in_context(ctx)
        output = net(data)
        prediction = nd.argmax(output, axis=1).astype(np.int32)
        num_instance += len(prediction)
        sum_metric += (prediction==label).sum()
    accuracy = (sum_metric.astype(np.float32)/num_instance)
    return accuracy.asscalar()

In [ ]:
%%time
print("Untrained network Test Accuracy: {0:.4f}".format(evaluate_accuracy_gluon(dataloader_test_ft, net)))

In [ ]:
from mxnet import gluon, nd, autograd

val_accuracy = 0
for epoch in range(15):
    for i, (data, label) in enumerate(dataloader_train_ft):
        data = data.astype(np.float32).as_in_context(ctx)
        label = label.as_in_context(ctx)

        if i%20==0 and i >0:
            print('Batch [{0}] loss: {1:.4f}'.format(i, loss.mean().asscalar()))

        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(data.shape[0])

    nd.waitall() # wait at the end of the epoch    
    new_val_accuracy = evaluate_accuracy_gluon(dataloader_test_ft, net)    
    print("Epoch [{0}] Test Accuracy {1:.4f} ".format(epoch, new_val_accuracy))

    # We perform early-stopping regularization, to prevent the model from overfitting
#     if val_accuracy > new_val_accuracy:
#         print('Validation accuracy is decreasing, stopping training')
#         break
    val_accuracy = new_val_accuracy

In [ ]:
#Save net parameters
current_model = 'edgeLite_ft'
# net1 = gluon.nn.HybridSequential()
# net1.hybridize()
# with net1.name_scope():
#     net1.add(net)

net.export(current_model, 10)
saved_model = current_model + '.params'
print (saved_model)
net.save_parameters(saved_model)

In [ ]:
def transform(img):
    return nd.array(np.expand_dims(np.transpose(img, (2,0,1)),axis=0).astype(np.float32), ctx=ctx)

In [ ]:
import os
import subprocess
import time
import cv2

#test images path
path = './data/sdir/'
script_path = '/usr/lib/python3/dist-packages/edgetpu/demo/'
#folders = []

files = os.listdir(path)
#print (files)

images = {}
false_assum = {}

for file in files:
   if 'hazard' in file:
      images.update({file : 'hazard'})
   elif 'clean' in file:
      images.update({file : 'clean'})

#print (images)

#total_time = time.time()
for key in images:
   print ('\n\nImage name : ',key, 'Tag: ' , images[key], 'floor and the classification result is : ' )
   start_time = time.time()
   img = cv2.imread(path+key)
   img = cv2.resize(img,(224,224))
   #img = np.reshape(img,[1,224,224,3])
   #image = tf.cast(img, tf.float32)
   image = transform(img)
   classes = net(image)
   print(classes)
   if classes[0][0] > classes[0][1]:
        res = 'clean'
        print('clean')
   else:
        res = 'hazard'
        print('hazard')
   
#    print("---%s seconds ---" % (time.time() - start_time))
   print("Image Name", key ,"classify as : ", res )
   if images[key] in res:
      images[key] = 1
   else:
      images[key] = 0
      false_assum.update({key: 0})
#print("---%s seconds ---" % (time.time() - total_time))

print ("List of wrong assumtion : ", false_assum)
print ('########## Final result #########')
print ('Total right assumtion : ', sum(images.values()), '\nTotal worng assumtion : ', len(images)-sum(images.values()), '\nModel accuracy = ', round((sum(images.values())*100)/len(images), 3), '%')